# KLOOK Sitemap URL Collector

**목적**: KLOOK Sitemap에서 URL만 수집하여 텍스트 파일로 저장  
**특징**: 최대한 단순한 구조, 중복 검사는 메인 크롤러에서 수행  
**출력**: `sitemap_urls/{도시명}_urls.txt`

---

In [ ]:
import os
import time
import requests
from datetime import datetime
from urllib.parse import urljoin, urlparse
import xml.etree.ElementTree as ET

def normalize_city_name(city_name):
    """도시명 정규화 (공백 제거, 소문자 변환 등)"""
    return city_name.strip()

# 기본 설정
CITY_NAME = normalize_city_name("도쿄")  # 수집할 도시명 변경
BASE_URL = "https://www.klook.com"

# Sitemap URL 패턴
SITEMAP_PATTERNS = {
    "도쿄": "https://www.klook.com/sitemap/activities/tokyo.xml",
    "방콕": "https://www.klook.com/sitemap/activities/bangkok.xml",
    "싱가포르": "https://www.klook.com/sitemap/activities/singapore.xml",
    "홍콩": "https://www.klook.com/sitemap/activities/hong-kong.xml",
    "타이베이": "https://www.klook.com/sitemap/activities/taipei.xml",
    "서울": "https://www.klook.com/sitemap/activities/seoul.xml",
    "오사카": "https://www.klook.com/sitemap/activities/osaka.xml",
    "교토": "https://www.klook.com/sitemap/activities/kyoto.xml"
}

# 출력 디렉토리 생성
os.makedirs("sitemap_urls", exist_ok=True)

print(f"대상 도시: {CITY_NAME}")
print(f"출력 폴더: sitemap_urls/")
print(f"시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n" + "="*50)

In [ ]:
# Cell 2: Sitemap URL 가져오기 함수
def get_sitemap_url(city_name):
    """도시명으로 Sitemap URL 반환"""
    return SITEMAP_PATTERNS.get(city_name)

def fetch_sitemap_xml(sitemap_url, max_retries=3):
    """Sitemap XML 가져오기 (재시도 포함)"""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/xml,application/xml,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive'
    }

    for attempt in range(max_retries):
        try:
            print(f"  Sitemap 요청 중... (시도 {attempt + 1}/{max_retries})")

            response = requests.get(sitemap_url, headers=headers, timeout=30)
            response.raise_for_status()

            print(f"  Sitemap 가져오기 성공 (크기: {len(response.content):,} bytes)")
            return response.text

        except requests.RequestException as e:
            print(f"  시도 {attempt + 1} 실패: {e}")
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 2
                print(f"  {wait_time}초 후 재시도...")
                time.sleep(wait_time)
            else:
                print(f"  Sitemap 가져오기 최종 실패")
                return None

def parse_sitemap_xml(xml_content):
    """XML에서 URL 목록 추출"""
    try:
        # 빈 내용 체크
        if not xml_content or not xml_content.strip():
            print("  빈 XML 내용")
            return []

        root = ET.fromstring(xml_content)

        # XML 네임스페이스 처리
        namespaces = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

        urls = []
        for url_elem in root.findall('.//ns:url', namespaces):
            loc_elem = url_elem.find('ns:loc', namespaces)
            if loc_elem is not None and loc_elem.text:
                url = loc_elem.text.strip()
                # KLOOK 활동 URL 필터링
                if '/activity/' in url and url.startswith('https://www.klook.com/'):
                    urls.append(url)

        print(f"  추출된 URL 개수: {len(urls):,}개")
        return urls

    except ET.ParseError as e:
        print(f"  XML 파싱 실패: {e}")
        return []
    except Exception as e:
        print(f"  URL 추출 실패: {e}")
        return []

print("Sitemap 처리 함수 준비 완료")

In [ ]:
# Cell 3: URL 저장 함수
def save_urls_to_file(urls, city_name):
    """URL 목록을 텍스트 파일로 저장"""
    if not urls:
        print("  저장할 URL이 없습니다.")
        return False

    # 중복 제거
    unique_urls = list(dict.fromkeys(urls))  # 순서 유지하면서 중복 제거
    duplicates_removed = len(urls) - len(unique_urls)

    if duplicates_removed > 0:
        print(f"  중복 URL 제거: {duplicates_removed}개")

    try:
        filename = f"sitemap_urls/{city_name}_urls.txt"

        with open(filename, 'w', encoding='utf-8') as f:
            for url in unique_urls:
                f.write(url + '\n')

        file_size = os.path.getsize(filename)
        print(f"  URL 파일 저장 완료!")
        print(f"     파일명: {filename}")
        print(f"     URL 개수: {len(unique_urls):,}개")
        print(f"     파일 크기: {file_size:,} bytes")

        return True

    except Exception as e:
        print(f"  파일 저장 실패: {e}")
        return False

def get_url_stats(city_name):
    """저장된 URL 파일 통계"""
    filename = f"sitemap_urls/{city_name}_urls.txt"

    if not os.path.exists(filename):
        return None

    try:
        with open(filename, 'r', encoding='utf-8') as f:
            urls = [line.strip() for line in f if line.strip()]

        file_size = os.path.getsize(filename)

        return {
            'filename': filename,
            'url_count': len(urls),
            'file_size': file_size,
            'sample_urls': urls[:5]  # 처음 5개 URL 샘플
        }

    except Exception as e:
        print(f"  파일 통계 확인 실패: {e}")
        return None

print("파일 저장 함수 준비 완료")

In [ ]:
# Cell 4: 메인 수집 실행
def collect_sitemap_urls(city_name):
    """지정된 도시의 Sitemap URL 수집 메인 함수"""
    print(f"\n'{city_name}' Sitemap URL 수집 시작")
    print("="*50)

    # 기존 파일 확인
    filename = f"sitemap_urls/{city_name}_urls.txt"
    if os.path.exists(filename):
        print(f"기존 파일이 존재합니다: {filename}")
        print(f"   계속 진행하면 덮어쓰기됩니다.")

    # 1. Sitemap URL 확인
    sitemap_url = get_sitemap_url(city_name)
    if not sitemap_url:
        print(f"'{city_name}'에 대한 Sitemap URL을 찾을 수 없습니다.")
        print(f"지원되는 도시: {', '.join(SITEMAP_PATTERNS.keys())}")
        return False

    print(f"Sitemap URL: {sitemap_url}")

    # 2. Sitemap XML 가져오기
    xml_content = fetch_sitemap_xml(sitemap_url)
    if not xml_content:
        return False

    # 3. URL 추출
    print(f"\nXML에서 URL 추출 중...")
    urls = parse_sitemap_xml(xml_content)

    if not urls:
        print(f"유효한 URL을 찾을 수 없습니다.")
        return False

    # 4. URL 파일로 저장
    print(f"\nURL 파일 저장 중...")
    success = save_urls_to_file(urls, city_name)

    if success:
        print(f"\n'{city_name}' URL 수집 완료!")
        return True
    else:
        print(f"\n'{city_name}' URL 수집 실패")
        return False

# 실행
start_time = time.time()
result = collect_sitemap_urls(CITY_NAME)
end_time = time.time()

print(f"\n" + "="*50)
print(f"총 소요 시간: {end_time - start_time:.1f}초")
print(f"수집 결과: {'성공' if result else '실패'}")
print(f"완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Cell 5: 수집 결과 확인
stats = get_url_stats(CITY_NAME)

if stats:
    print(f"'{CITY_NAME}' URL 파일 통계:")
    print(f"   파일명: {stats['filename']}")
    print(f"   URL 개수: {stats['url_count']:,}개")
    print(f"   파일 크기: {stats['file_size']:,} bytes")

    print(f"\n샘플 URL (처음 5개):")
    for i, url in enumerate(stats['sample_urls'], 1):
        print(f"   {i}. {url}")

    print(f"\nURL 수집 파일이 준비되었습니다!")
    print(f"   다음 단계: 메인 크롤러(KLOOK_Crawler_v2.ipynb)에서 이 파일을 사용하세요.")

else:
    print(f"'{CITY_NAME}'의 URL 파일을 찾을 수 없습니다.")
    print(f"   Cell 4를 먼저 실행하여 URL을 수집하세요.")

---

## 사용법

1. **Cell 1**: `CITY_NAME` 변경 후 실행
2. **Cell 2-3**: 함수 정의 (실행만)
3. **Cell 4**: 메인 수집 실행
4. **Cell 5**: 결과 확인

## 출력 파일
- `sitemap_urls/{도시명}_urls.txt`
- 한 줄에 하나씩 URL 저장
- 메인 크롤러에서 이 파일을 읽어서 처리

## 저장 위치
```
/mnt/c/Users/redsk/OneDrive/デスクトップ/mikael_project/klook/
└── sitemap_urls/
    ├── 도쿄_urls.txt
    ├── 방콕_urls.txt  
    └── 싱가포르_urls.txt
```

---